# Курс "Программирование на языке Python. Уровень 4. Анализ и визуализация данных на языке Python. Библиотеки numpy, pandas, matplotlib"

## Модуль 6. Агрегирование и групповые операции в pandas

1. Составной (многоуровневый) индекс, горизонтальное "раскатывание" и "штабелирование" данных
1. Группировка данных
2. Агрегирование данных
3. Групповые операции над данными
4. Групповые операции и статистический анализ
5. Сводные таблицы
6. Визуализация данных средствами pandas
7. Сохранение данных



In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline

## "Раскатывание" и "штабелирование" данных, многоуровневый индекс

Индекс (как вертикальный, так и горизонтальный) в ```pandas``` может быть многоуровневым: он может состоять из нескольких вложенных уровней. Индексу можно дать имя при создании:

In [ ]:
pd.Index(['Ohio', 'Colorado'], name='state')

Работа со сложным индексом: функции ```stack()``` и ```unstack()```.

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

 Функция ```stack()``` "сложит" данные вертикально, создав ```Series``` с "многоуровневым индексом":

In [ ]:
stacked = data.stack()
stacked

Функция ```unstack()``` выполнит противоположную задачу - данные из "сложного индекса" вынесет, "раскатает" в колонки:

In [ ]:
stacked.unstack()

## Функции melt() и pivot()

Эти функции работают аналогично ```stack()``` и ```unstack()```, но они не формируют "сложный индекс", а выстраивают данные в матрице "в столбик" и наоборот, соответственно.

```melt()``` - преобразует столбцы в строки, добавляя соотвествующие столбцы variable и value. \
```pivot()```- наоборот, собирает данные по строкам в столбцы

In [ ]:
df = pd.DataFrame({'Function': ['Accounting', 'Administrative', 'IT Support'],
                   'Jan': [1, 2, 3],
                   'Feb': [4, 5, 6],
                   'Mar': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['Function'], var_name='Months', value_name='Values')
melted

In [ ]:
reshaped = pd.pivot(melted, index=['Function'], columns=['Months'], values='Values')
reshaped

__ПРАКТИКА__

Загрузите датасет "toy_budget.xlsx". Он содержит информацию по доходам и расходам подразделений компании по месяцам. Преобразуйте его в датасет, в котором месяцы отложены как столбцы и упорядочены так, как это задано в списке ```months```. 

Указание: используйте функцию ```reindex()``` для упорядочивания колонок.

In [ ]:
!conda install -y openpyxl

In [ ]:
months = ['apr',
 'may',
 'jun',
 'jul',
 'aug',
 'sep',
 'oct',
 'nov',
 'dec',
 'jan',
 'feb',
 'mar',]

df_toy_budget = pd.read_excel( 'data/toy_budget.xlsx', engine='openpyxl' )
df_toy_budget

In [ ]:
# ваш код здесь


## Группировка данных

Под группировкой будем понимать разделение объекта pandas на части по одному или нескольким признакам. Такое объединение данных применяется для:
 - вычисления групповых статистик (минимум, максимум, сумма, и т.д.)
 - применения групповых преобразований
 - вычисления сводных таблиц и кросстабулирования

В pandas для группировки используется функция ```groupby()```. Эта функция сама по себе ничего не вычисляет,кроме промужуточных данных о ключах, по которым производится объедиение. Функция возвращает объект, у которого есть методы для агрегатных вычислений.
 

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

In [ ]:
df['key1']

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [ ]:
grouped.mean()

Для группировки можно использовать несколько объектов Series:

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

...или выполнять группировку по всему DataFrame'у, передав ключи в виде списка:

In [ ]:
# это выражение должно вывести тот же результат, что и выражение выше
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

...также можно указать один ключ. Обратите внимание: куда пропал ключ 'key2'?

In [ ]:
means = df.groupby('key1').mean()
means

Группировку можно проводить по любой оси. Например, сгруппируем данные по типу:

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)
grouped.size()

Объект SeriesGroupBy поддерживает итерирование. Так, группы можно обходить в цикле ```for```. При этом итератор возвращает значение ключа и объект DataFrame группы:

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

__ЗАДАНИЕ__ Сгруппируйте датафрейм df по признакам key1 и key2 и обратите внимание, что вернет первый параметр итератора.

In [ ]:
# ваш код здесь


Также для группировки можно использовать словари и массивы. Рассмотрим пример:

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Alex', 'Jim', 'Travis'])
people

In [ ]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [ ]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

Также можно использовать для группировки функцию (например, ```len()```):

In [ ]:
people.groupby(len).sum()

## Создание множественного индекса

DataFrame с множественным индексом можно задать явно:

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    ['Specialized', 'Santa Cruz', 'GT', 'Fuji', 'Ninjago']],
                                    names=['country', 'brand'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

Множественный индекс удобно использовать для группировки данных. Можно указать уровень в параметре ```level=```.

In [ ]:
hier_df.groupby(level='brand', axis=1).sum()

### Агрегирование данных

Список оптимизированных агрегатных функций pandas:

- ```count()``` - количество отличных от NaN значений в группе
- ```sum()``` - сумма
- ```mean()``` - среднее (также по отличным от NaN значениям)
- ```median()``` - медиана (тоже по отличным от NaN)
- ```var()``` - дисперсия
- ```min()``` - минимум
- ```max()``` - максимум
- ```prod()``` - произведение
- ```first()``` - первый отличный от NaN в группе
- ```last()``` - последний отличный от NaN в группе

Также можно написать свою собственную функцию и передать ее в функцию ```agg()```. Она будет принимать на вход объект Series и возращать скалярное значение.


In [ ]:
df

In [ ]:
def mean_median(arr):
    return arr.mean() - arr.median()
df.groupby('key1').agg(mean_median)

Также в ```agg()``` можно передавать список функций, стандартные функции передаются по именам:

In [ ]:
df.groupby('key1').agg(['mean', 'median', mean_median])

In [ ]:
df.groupby('key1').agg([("Среднее", 'mean'), ("Медиана", 'median'), ("Среднее-медиана", mean_median)])

__ЗАДАНИЕ__: Для набора данных "чаевые" посчитайте следующее:
 - общую сумму чаевых за каждый день
 - средний процент для групп в зависимости от пола, а потом и дня
 - выведите минимальный и максимальный процент чаевых в зависимости от дня и времени

In [ ]:
tips = pd.read_csv('data/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

In [ ]:
# ваш код здесь



### Групповые операции над данными

Функция apply позволяет применить заданную функцию к каждой группе. И более того, она может вернуть векторное значение.

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]
top(tips, n=6)

Так мы можем вывести самых щедрых на чаевые из групп "курящие/некурящие":

In [ ]:
tips.groupby('smoker').apply(top)

Параметры к нашей функции можно передать через ту же функцию ```apply()```

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

Избавиться от индекса в выводе данных можно с помощью параметра ```group_keys=False```

In [ ]:
tips.groupby(['smoker', 'day'], group_keys=False).apply(top, n=1, column='total_bill')

__ЗАДАНИЕ__: В заданном ниже датасете посчитайте средний балл по фамилиям студентов:

In [ ]:
students = ['Вася Иванов', 'Витя Петров', 'Таня Текслер', 'Игорь Иванов' \
            , 'Петр Текслер', 'Игорь Иванов', 'Андрей Иванов', 'Ольга Петрова']
marks = [4, 4, 5, 3, 4, 5, 5, 3]
df_marks = pd.DataFrame( {'student': students, 'mark': marks} )

# ваш код здесь

### Групповые операции при статистическом анализе

Пример для вычисления группового взвешенного среднего.

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
def get_wavg(g):
    return np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

Кореляция.

In [ ]:
close_px = pd.read_csv('data/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.head()

In [ ]:
rets = close_px.pct_change().dropna() # сбросим незаполненные данные и посчитаем изменения в процентном отношении
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Сводные таблицы

Для построения сводных таблиц используется метод ```pivot_table()```. По умолчанию она считает средние по каждому показателю.

Рассмотрим на примере "чаевых":

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'],
                 columns='smoker', margins=True, fill_value=0)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

__ЗАДАНИЕ__: постройте сводную таблицу с суммами чаевых в зависимости от дня недели, времени и пола.

In [ ]:
# ваш код здесь


#### Кросстабуляция

или таблица сопряженности - частный случай сводной таблицы для подсчета групповых частот.

In [ ]:
party_counts = pd.crosstab(tips['day'], tips['size'])# Not many 1- and 6-person parties
print(party_counts)

__ЗАДАНИЕ__ Реализуйте ту же задачу, но функцией ```pivot_table()```.

In [ ]:
# ваш код здесь

__Визуализация данных средствами Pandas__

Рассмотрим базовые функции визуализации в pandas на примере датасета моделей playboy с 1953 по 2009 года.

In [ ]:
df_playboy = pd.read_csv('data/girls.csv')
df_playboy.head()

"Не выходя" из Pandas, можно, например, строить гистограммы по одному или нескольким показателям:

In [ ]:
df_playboy['Height'].hist()

In [ ]:
df_playboy[['Weight', 'Height']].hist()

Можно построить несколько гистограмм на одном поле:

In [ ]:
df_playboy[['Bust', 'Waist', 'Hips']].plot.hist(alpha=0.5)

Можно построить матрицу диаграмм рассеяния, воспользовавшись подмодулем ```plotting``` и ```функцией scatter_matrix()```.

In [ ]:
_ = pd.plotting.scatter_matrix(df_playboy[['Bust', 'Waist', 'Hips', 'Height', 'Weight']], 
                  figsize=(15, 15))

__ПРАКТИКА__

Есть предположение, что с середины 80-х годов моделями были в основвном высокие и очень высокие девушки. Постройте с помощью функции plot график зависимости роста модели от года. 

_ПОДСКАЗКА_: сгруппируйте датасет по признаку "год" ("Year") и возьмите средний рост (признак "Height") модели.

Пример анализа данных на этом датасете: решение задачи поиска аномалий, в статье Ю. Кашницкого на "Хабре": https://habr.com/ru/post/251225/

In [ ]:
# ваш код здесь


### Сохранение данных

Для сохранения датасетов в требуемом виде можно использовать следующие функции:
 - ```to_csv()``` - для сохранения данных в виде CSV
 - ```to_excel()``` - для сохранения данных в виде Excel Workbook \
...а также во можестве других форматов (см. документацию).
 
Также данные можно экспортировать в структуры Python и numpy:
 - ```to_dict()``` - этот метод вернет словарь с содержимым DataFrame, ровно в том же виде, чтобы из него можно было бы создать новый DataFrame
 - ```to_dict('records')``` - в данном случае этот метод вернет список словарей
 - ```to_numpy()``` - а этот метод можно использовать, если вам нужна матрица numpy

__ПРАКТИКА__

В преобразованном dataset'е ```df_toy_budget``` (см. предыдущее задание из блока "раскатывание"/"штабелирование") подсчитайте суммы по каждому месяцу и запишите их в файл ```toy_budget_monthly.xlsx```.

In [ ]:
# ваш код здесь
